## Clear Kaggle working dir

In [1]:
# import shutil
# import os

# # Répertoire de travail Kaggle
# working_dir = "/kaggle/working"

# # Parcours tous les fichiers et dossiers dans /kaggle/working
# for filename in os.listdir(working_dir):
#     file_path = os.path.join(working_dir, filename)
#     try:
#         # Supprime fichier ou dossier
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)  # Supprimer fichier ou lien symbolique
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)  # Supprimer dossier récursivement
#     except Exception as e:
#         print(f"Erreur lors de la suppression de {file_path}: {e}")

# print("Kaggle working cleaned")

## Install Necessary dependencies

In [2]:
!pip install evaluate rouge_score json-repair
!pip install orkg requests_toolbelt urllib3 pdfplumber
!pip uninstall -y keycloak
!pip install python-keycloak==1.7.0
print("\nSuccessfull dependance installation\n")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0bf25206a5e561de03f61a69ba9b06128ed0b1a7818be3e978175edd5daae55e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which 

## Disable Wandb reporting during finetuning

In [3]:
# Disable wand db option
import os

os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

print("Disable wandb option")

Disable wandb option


## Define prompt template: 
- Zero-shot
- Few-shot
- Role-prompting
- Contextual prompting

In [4]:
import json

# progression des champs
schemas = [
    {"local name": "string"},
    {"local name": "string", "common name": "string"},
    {"local name": "string", "common name": "string", "geographical area": "string"},
    {"local name": "string", "common name": "string", "geographical area": "string", "food group": "string"},
    {
        "local name": "string",
        "common name": "string",
        "geographical area": "string",
        "food group": "string",
        "ingredients": [
            {"common name": "string", "scientific name": "string"}
        ]
    },
    {
        "geographical area": "string",
        "local name": "string",
        "common name": "string",
        "food group": "string",
        "ingredients": [
            {"common name": "string", "scientific name": "string"}
        ],
        "food components": [
            {"label": "string", "value": "float", "unit": "string"}
        ]
    }
]

# exemples pour few-shot
examples = [
    {
        "text": """CONDENSED FOOD COMPOSITION TABLE | TABLE DE COMPOSITION DES ALIMENTS CONDENSÉE\n423\nFOOD FOOD NAME CHOLESTEROL SAT FA MONOUNSAT FA POLYUNSAT FA LINOLEIC ACID ALPHA-LINOLENIC ACID PHYTATE\nCODE IN ENGLISH (mg) (g) (g) (g) (g) (g) (mg)\nINFOODS TAGNAMES CHOLE(mg) FASAT(g) FAMS(g) FAPU(g) F18D2CN6(g) F18D3CN3(g) PHYTCPP(mg)\nSoups and sauces/Soupes et sauces\nNayoungn vand zindo (Burkina Faso)*: sweet potato leaf\n14_009 8 6.04 4.52 1.40 [1.20] [0.04] [2]\nsauce, fish, red palm oil and vegetables\nPésgo né tomate zéindo (Burkina Faso)*: sauce with lamb,\n14_002 19 4.41 5.64 5.12 4.87 0.16 [3]\ntomato and vegetables\nSiikam zéédo (Burkina Faso)*: groundnut sauce with\n14_006 6 3.44 4.25 2.22 2.05 0.05 [33]\nvegetables, fish and fermented African locust beans\nSiné zindo (Burkina Faso)*: sesame sauce with fish and\n14_028 8 2.77 5.87 8.24 7.99 0.07 [135]\nvegetables\nSoup from chicken, beer yeast, vegetables and fermented\n14_033 11 1.08 2.03 1.95 1.90 0.04 [1]\nAfrican locust beans (Burkina Faso)*\n14_034 Soup with cabbage and vegetables (Burkina Faso)* 0 0.04 0.03 0.04 0.03 0.01 [3]\nTéi kam zéédo (Burkina Faso)*: red palm oil sauce with\n14_005 7 2.95 3.94 2.11 1.95 0.02 [33]\nfish and vegetables\nTéi né maane zéindo (Burkina Faso)*: oil palm fruit sauce\n14_024 6 2.77 2.83 0.80 0.09 0.01 [1]\nwith okra powder, fish and vegetables\nTéi zéindo (Burkina Faso)*: simple oil palm fruit sauce with\n14_023 3 6.47 6.85 1.69 0.18 0.01 [0]\nfish and vegetables\nToeg koing zéédo (Burkina Faso)*: dried baobab leaf sauce\n14_020 5 0.50 0.69 1.09 0.95 0.04 [1]\nwith fish, vegetables and fermented African locust beans\nToeg maas zéédo (Burkina Faso)*: fresh baobab leaf sauce\n14_022 6 0.74 1.20 1.87 1.71 0.04 [7]\nwith fish, vegetables and fermented African locust beans\n14_010 Yassa sauce with lamb, onion and mustard (Burkina Faso)* 10 4.09 7.00 8.46 8.21 0.19 [3]""",
        "output": """[{"geographical area": "West african", "local name": "Onion", "common name": null, "food group": "Vegetables and their products", "food components": [{"label": "Onion,raw01_energy", "value": 37.0, "unit": "Kcal"}, {"label": "Onion,raw01_water", "value": 89.5, "unit": "g"}, {"label": "Onion,raw01_protein", "value": 1.1, "unit": "g"}, {"label": "Onion,raw01_fat", "value": 0.1, "unit": "g"}, {"label": "Onion,raw01_carbohydrate-available", "value": 6.9, "unit": "g"}, {"label": "Onion,raw01_fibre", "value": 1.8, "unit": "g"}, {"label": "Onion,raw01_ash", "value": 0.6, "unit": "g"}, {"label": "Onion,raw01_calcium", "value": 25.0, "unit": "mg"}, {"label": "Onion,raw01_iron", "value": 0.3, "unit": "mg"}, {"label": "Onion,raw01_magnesium", "value": 10.0, "unit": "mg"}, {"label": "Onion,raw01_phosphorus", "value": 39.0, "unit": "mg"}, {"label": "Onion,raw01_potassium", "value": 183.0, "unit": "mg"}, {"label": "Onion,raw01_sodium", "value": 4.0, "unit": "mg"}, {"label": "Onion,raw01_zinc", "value": 0.26, "unit": "mg"}, {"label": "Onion,raw01_copper", "value": 0.04, "unit": "mg"}, {"label": "Onion,raw01_b carotene equivalent", "value": 1.0, "unit": "mcg"}, {"label": "Onion,raw01_vitamin E", "value": 0.04, "unit": "mg"}, {"label": "Onion,raw01_thiamin", "value": 0.05, "unit": "mg"}, {"label": "Onion,raw01_riboflavin", "value": 0.04, "unit": "mg"}, {"label": "Onion,raw01_niacin", "value": 0.2, "unit": "mg"}, {"label": "Onion,raw01_vitamin B6", "value": 0.1, "unit": "mg"}, {"label": "Onion,raw01_folate", "value": 16.0, "unit": "mcg"}, {"label": "Onion,raw01_vitamin C", "value": 10.3, "unit": "mg"}]}, {"geographical area": "West african", "local name": "Onion", "common name": null, "food group": "Vegetables and their products", "food components": [{"label": "Onion,boiled01_energy", "value": 45.0, "unit": "Kcal"}, {"label": "Onion,boiled01_water", "value": 87.2, "unit": "g"}, {"label": "Onion,boiled01_protein", "value": 1.4, "unit": "g"}, {"label": "Onion,boiled01_fat", "value": 0.1, "unit": "g"}, {"label": "Onion,boiled01_carbohydrate-available", "value": 8.4, "unit": "g"}, {"label": "Onion,boiled01_fibre", "value": 2.2, "unit": "g"}, {"label": "Onion,boiled01_ash", "value": 0.8, "unit": "g"}, {"label": "Onion,boiled01_calcium", "value": 29.0, "unit": "mg"}, {"label": "Onion,boiled01_iron", "value": 0.3, "unit": "g"}, {"label": "Onion,boiled01_magnesium", "value": 7.0, "unit": "mg"}, {"label": "Onion,boiled01_phosphorus", "value": 43.0, "unit": "mg"}, {"label": "Onion,boiled01_potassium", "value": 101.0, "unit": "mg"}, {"label": "Onion,boiled01_sodium", "value": 3.0, "unit": "mg"}, {"label": "Onion,boiled01_zinc", "value": 0.24, "unit": "mg"}, {"label": "Onion,boiled01_copper", "value": 0.05, "unit": "mg"}, {"label": "Onion,boiled01_b ccarotene equivalent", "value": 1.0, "unit": "mcg"}, {"label": "Onion,boiled01_vitamin E", "value": 0.05, "unit": "mg"}, {"label": "Onion,boiled01_thiamin", "value": 0.04, "unit": "mg"}, {"label": "Onion,boiled01_riboflavin", "value": 0.04, "unit": "mg"}, {"label": "Onion,boiled01_niacin", "value": 0.2, "unit": "mg"}, {"label": "Onion,boiled01_vitamin B6", "value": 0.08, "unit": "mg"}, {"label": "Onion,boiled01_folate", "value": 10.0, "unit": "mcg"}, {"label": "Onion,boiled01_vitamin C", "value": 4.2, "unit": "mg"}]}, {"geographical area": "West african", "local name": "Onion", "common name": null, "food group": "Vegetables and their products", "food components": [{"label": "Onion,died01_energy", "value": 354.0, "unit": "Kcal"}, {"label": "Onion,died01_water", "value": 4.2, "unit": "g"}, {"label": "Onion,died01_protein", "value": 9.6, "unit": "g"}, {"label": "Onion,died01_fat", "value": 1.1, "unit": "g"}, {"label": "Onion,died01_carbohydrate-available", "value": 71.1, "unit": "mg"}, {"label": "Onion,died01_fibre", "value": 10.7, "unit": "g"}, {"label": "Onion,died01_ash", "value": 3.4, "unit": "g"}, {"label": "Onion,died01_calcium", "value": 239.0, "unit": "mg"}, {"label": "Onion,died01_iron", "value": 2.1, "unit": "mg"}, {"label": "Onion,died01_magnesium", "value": 64.0, "unit": "mg"}, {"label": "Onion,died01_phosphorus", "value": 282.0, "unit": "mg"}, {"label": "Onion,died01_potassium", "value": 1510.0, "unit": "mg"}, {"label": "Onion,died01_sodium", "value": 21.0, "unit": "mg"}, {"label": "Onion,died01_zinc", "value": 1.8, "unit": "mg"}, {"label": "Onion,died01_copper", "value": 0.4, "unit": "mg"}, {"label": "Onion,died01_vitamin A-RAE", "value": 1.0, "unit": "mcg"}, {"label": "Onion,died01_b carotene equivalent", "value": 11.0, "unit": "mcg"}, {"label": "Onion,died01_vitamin E", "value": 0.4, "unit": "mg"}, {"label": "Onion,died01_thiamin", "value": 0.37, "unit": "g"}, {"label": "Onion,died01_riboflavin", "value": 0.1, "unit": "mg"}, {"label": "Onion,died01_niacin", "value": 0.9, "unit": "mg"}, {"label": "Onion,died01_vitamin B6", "value": 1.6, "unit": "mg"}, {"label": "Onion,died01_folate", "value": 166.0, "unit": "mcg"}, {"label": "Onion,died01_vitamin C", "value": 54.0, "unit": "mg"}]}]"""
    },
]

prompts = {"prompts": []}

# V1 → V6 : zero-shot
for i, schema in enumerate(schemas, start=1):
    prompts["prompts"].append({
        "version": i,
        "mode": "zero-shot",
        "instruction": "Extract all food contributions mentioned in the following raw tabular data text. Each contribution must be returned as a valid JSON object with the following structure:",
        "schema": schema
    })

# V7 → V12 : few-shot
for i, schema in enumerate(schemas, start=7):
    prompts["prompts"].append({
        "version": i,
        "mode": "few-shot",
        "instruction": "Extract all food contributions from raw tabular data text using the examples below as guidance.",
        "examples": examples,
        "schema": schema
    })

# V13 → V24 : role prompting
for i, schema in enumerate(schemas * 2, start=13):  # répète 6 fois zero-shot + 6 fois few-shot
    mode = "role-zero-shot" if i < 19 else "role-few-shot"
    entry = {
        "version": i,
        "mode": mode,
        "role": "You are an expert in food composition analysis.",
        "schema": schema
    }
    if "few-shot" in mode:
        entry["instruction"] = "Your task is to extract all food contributions from the raw tabular data text using the examples below as guidance."
        entry["examples"] = examples
    else:
        entry["instruction"] = "Your task is to extract all food contributions mentioned in the following raw tabular data text. Return them as valid JSON objects with the following structure:"
    prompts["prompts"].append(entry)

# Sauvegarde du fichier
with open("/kaggle/working/prompts.json", "w") as f:
    json.dump(prompts, f, indent=2)

print("prompts.json généré avec 24 versions")


import json

def load_prompt(version: int, input_text: str, path: str = "/kaggle/working/prompts.json") -> str:
    # Charger le fichier
    with open(path, "r") as f:
        data = json.load(f)

    # Chercher la version
    prompt_data = next((p for p in data["prompts"] if p["version"] == version), None)
    if prompt_data is None:
        raise ValueError(f"Version {version} not found in {path}")

    # Construire le corps du prompt
    base = ""

    # Ajouter rôle si présent
    if "role" in prompt_data:
        base += prompt_data["role"] + "\n\n"

    # Ajouter instruction
    base += prompt_data["instruction"] + "\n\n"

    # Ajouter exemples si présents (few-shot ou role-few-shot)
    if "examples" in prompt_data:
        base += "Examples:\n"
        for ex in prompt_data["examples"]:
            base += f"Text: {ex['text']}\nOutput: {json.dumps(ex['output'], indent=2)}\n\n"

    # Ajouter schema
    base += "Schema:\n" + json.dumps(prompt_data["schema"], indent=2) + "\n\n"

    # Ajouter le texte cible
    base += f"Text:\n{input_text}\n\nJSON array:"

    return base

print('\nPrompt Loader function defined successfully')

prompts.json généré avec 24 versions

Prompt Loader function defined successfully


In [5]:
prompt_test = load_prompt(6, "myText")
print(prompt_test)

Extract all food contributions mentioned in the following raw tabular data text. Each contribution must be returned as a valid JSON object with the following structure:

Schema:
{
  "geographical area": "string",
  "local name": "string",
  "common name": "string",
  "food group": "string",
  "ingredients": [
    {
      "common name": "string",
      "scientific name": "string"
    }
  ],
  "food components": [
    {
      "label": "string",
      "value": "float",
      "unit": "string"
    }
  ]
}

Text:
myText

JSON array:


In [6]:
# Utils: Get prompt template

# def get_prompt(input_text):
#     # input_text = " ".join(input_text)  # fusionne les segments s’il y en a plusieurs
#     input_text = input_text.strip()


#     # === Prompt 1 : Zero-shot ===
#     zero_shot = f"""Extract all food contributions mentioned in the following text.

#         Each contribution must be returned as a JSON object with the following structure:
        
#         {{
#           "geographical area": "string",
#           "local name": "string",
#           "common name": "string",
#           "food group": "string",
#           "ingredients": [
#             {{
#               "common name": "string",
#               "scientific name": "string"
#             }}
#           ],
#           "food components": [
#             {{
#               "label": "string",
#               "value": float,
#               "unit": "string"
#             }}
#           ]
#         }}

#         If a field is not present in the text or you're not sure about its value, set it to `unanswerable`. Do not guess or hallucinate.
        
#         Return a JSON array containing all contributions.
        
#         Text:
#         {input_text}

#         JSON array:
#         """

#     # === Prompt 2 : Few-shot ===
#     few_shot = f"""You are an expert food knowledge extraction model.

#         You will read a text and extract a list of food contributions. Each contribution must follow this structure:
        
#         {{
#           "geographical area": "string",
#           "local name": "string",
#           "common name": "string",
#           "food group": "string",
#           "ingredients": [
#             {{
#               "common name": "string",
#               "scientific name": "string"
#             }}
#           ],
#           "food components": [
#             {{
#               "label": "string",
#               "value": float,
#               "unit": "string"
#             }}
#           ]
#         }}

#         If a field is not present in the text or you're not sure about its value, set it to `unanswerable`. Do not guess or hallucinate.
        
#         Return a valid JSON array containing one or more contributions.
        
#         Example 1:
#         Text:
#         Cassava is boiled and served with palm oil. It contains Vitamin C (14 mg) and Water (60%).
        
#         Answer:
#         [
#           {{
#             "common_name": "Cassava",
#             "food_form": "Boiled",
#             "food_name": "Boiled cassava",
#             "ingredients": [{{ "common_name": "Palm oil", "scientific_name": null }}],
#             "food_components": [
#               {{ "label": "Vitamin C", "value": 14.0, "unit": "mg" }},
#               {{ "label": "Water", "value": 60.0, "unit": "%" }}
#             ]
#           }}
#         ]
        
#         Example 2:
#         Text:
#         Roasted maize is eaten with coconut. It contains Energy (250 kcal).
        
#         Answer:
#         [
#           {{
#             "common_name": "Maize",
#             "food_form": "Roasted",
#             "food_name": "Roasted maize with coconut",
#             "ingredients": [{{ "common_name": "Coconut", "scientific_name": null }}],
#             "food_components": [
#               {{ "label": "Energy", "value": 250.0, "unit": "kcal" }}
#             ]
#           }}
#         ]
        
#         Now extract the contributions from the following text:
        
#         Text:
#         {input_text}

#         Answer:
#         """

#     # === Prompt 3 : Role prompting ===
#     role_prompt = f"""You are a food composition scientist specialized in extracting structured data from articles.

#         Your task is to read a text and extract a list of food contributions. Each contribution must follow this JSON format:
        
#         {{
#           "geographical area": "string",
#           "local name": "string",
#           "common name": "string",
#           "food group": "string",
#           "ingredients": [
#             {{
#               "common name": "string",
#               "scientific name": "string"
#             }}
#           ],
#           "food components": [
#             {{
#               "label": "string",
#               "value": float,
#               "unit": "string"
#             }}
#           ]
#         }}

#         If a field is not present in the text or you're not sure about its value, set it to `unanswerable`. Do not guess or hallucinate.
        
#         Make sure to output a valid JSON array.
        
#         Text:
#         {input_text}

#         JSON Array:
#         """

#     # === Prompt 4 : Contextual prompting ===
#     contextual = f"""Context:
#         You are analyzing a scientific document describing foods composition tables. Your goal is to extract structured food composition data for use in a food knowledge graph.
        
#         Expected output format:
#         Each contribution must be returned as a JSON object with this structure:
        
#         {{
#           "geographical area": "string",
#           "local name": "string",
#           "common name": "string",
#           "food group": "string",
#           "ingredients": [
#             {{
#               "common name": "string",
#               "scientific name": "string"
#             }}
#           ],
#           "food components": [
#             {{
#               "label": "string",
#               "value": float,
#               "unit": "string"
#             }}
#           ]
#         }}

#         If a field is not present in the text or you're not sure about its value, set it to `unanswerable`. Do not guess or hallucinate.
        
#         Return a JSON array of one or more contributions.
        
#         Text:
#         {input_text}

#         JSON array:
#         """

#     # Choix aléatoire
#     prompts = [zero_shot, few_shot, role_prompt, contextual]
#     # return random.choice(prompts)
#     return zero_shot

# print("\nget_prompt defined successfully")

In [7]:
# Evaluation

import json
import re
from collections import Counter
from enum import Enum

import evaluate
import numpy
from fuzzywuzzy import fuzz

numpy.random.seed(42)
unanswerable = 'unanswerable'
feature_names = ["geoographical area", "local name", "common name", "food group", "ingredients", "food components"]

class MatchType(Enum):
    EXACT = "EXACT"
    PARTIAL = "PARTIAL"


def is_answerable(text):
    return str(text).strip().lower() != unanswerable


def is_unanswerable(text):
    return str(text).strip().lower() == unanswerable


def calculate_fuzz_ratio(text1, text2):
    return fuzz.ratio(str(text1).strip().lower(), str(text2).strip().lower())


def get_contribution_feature_values_list(contribution):
    return str(contribution).strip().split("\n")


def is_partial_match_json_based(label_feature_value, prediction_feature_value):
    # print("\nis_partial_match_json_based\n")
    # print(f"label_feature_value: {label_feature_value}")
    # print(f"prediction_feature_value: {prediction_feature_value}\n")
    # print(f"fuzzy ration: {calculate_fuzz_ratio(label_feature_value, prediction_feature_value)}\n")
    
    return label_feature_value != '-' and prediction_feature_value != '-' and label_feature_value != "" and prediction_feature_value != "" \
        and calculate_fuzz_ratio(label_feature_value, prediction_feature_value) >= 80


def is_exact_match_json_based(label_feature_value, prediction_feature_value):
    return label_feature_value != '-' and prediction_feature_value != '-' \
        and label_feature_value != "" and prediction_feature_value != "" \
        and str(label_feature_value).strip().lower() == str(prediction_feature_value).strip().lower()


def get_number_of_values_of_feature_json_based(feature_name, json_item_list):
    number_of_features_with_value = sum(
        [1 for contribution in json_item_list if has_feature_value_json_based(contribution, feature_name)])
    # print("in this text ", feature_name, " has ", number_of_features_with_value, " occurrence with values ")
    return number_of_features_with_value

def has_feature_value_json_based(contribution, feature_name):
    contribution_root = get_contribution_root(contribution)
    feature_value = get_feature_value_json_based(contribution_root, feature_name)
    # contribution_root = contribution.get("contribution", {}) if isinstance(contribution, dict) else {}
    # feature_value = contribution_root.get(feature_name, {}) if isinstance(contribution_root, dict) else {}
    return feature_value != '-' and feature_value != ""


def get_feature_value_json_based(contribution_root, feature_name):
    # feature_value = ""
    # if isinstance(contribution_root, dict):
    #     for key in contribution_root.keys():
    #         if key.strip().lower() == feature_name.lower():
    #             feature_value = str(contribution_root[key]).strip().lower()
    #             break
    # return feature_value
    feature_value = ""
    if not isinstance(contribution_root, dict):
        return feature_value

    for key in contribution_root.keys():
        if key.strip().lower() == feature_name.lower():
            value = contribution_root[key]

            # Si c’est une liste de dicts, on convertit en JSON trié
            if isinstance(value, list) and all(isinstance(x, dict) for x in value):
                feature_value = json.dumps(value, sort_keys=True)
            else:
                feature_value = str(value).strip().lower()
            break

    return feature_value


def get_contribution_root(contribution):
    """
    Pour ta structure, on considère que le dictionnaire de contribution
    est déjà à la racine. Donc on retourne directement l'objet s'il est un dict.
    """
    return contribution if isinstance(contribution, dict) else {}

# ---------------------------------------------------------------------------------------------------------------------------


# def extract_label_prediction_feature_values_json(feature_name, label_contribution, prediction_contribution):
#     label_feature_value = label_contribution.get("contribution", {}).get(feature_name, "") if isinstance(
#         label_contribution, dict) else ""
#     prediction_contribution_root = get_contribution_root(contribution=prediction_contribution)
#     prediction_feature_value = get_feature_value_json_based(contribution_root=prediction_contribution_root,
#                                                             feature_name=feature_name)
#     return label_feature_value, prediction_feature_value
def extract_label_prediction_feature_values_json(feature_name,label_contribution, prediction_contribution ):
    # print(f"label contribution: {label_contribution}")
    if not isinstance(label_contribution, dict) or not isinstance(prediction_contribution, dict):
        return "", ""

    if feature_name in ["geographical area", "local name", "common name", "food group"]:
        label_value = label_contribution.get(feature_name, "").lower()
        prediction_value = prediction_contribution.get(feature_name, "").lower()

    elif feature_name == "ingredients":
        label_ingredients = label_contribution.get("ingredients", [])
        prediction_ingredients = prediction_contribution.get("ingredients", [])
        label_value = sorted([i.get("common name", "").lower() for i in label_ingredients])
        prediction_value = sorted([i.get("common name", "").lower() for i in prediction_ingredients])

    elif feature_name == "food components":
        label_components = label_contribution.get("food components", [])
        prediction_components = prediction_contribution.get("food components", [])
        label_value = sorted(
            [(c.get("label", "").lower(), float(c.get("value", 0)), c.get("unit", "").lower()) for c in label_components]
        )
        prediction_value = sorted(
            [(c.get("label", "").lower(), float(c.get("value", 0)), c.get("unit", "").lower()) for c in prediction_components]
        )
    else:
        label_value, prediction_value = "", ""

    return label_value, prediction_value
    

def compute_exact_score_between_2_contributions_json_based(prediction_contribution, label_contribution, feature_name):

    # print("compute_exact_score_between_2_contributions_json_based - INPUT \n")
    # print("prediction_contribution ----")
    # print(prediction_contribution)
    # print("\n\n")

    # print("label_contribution ----")
    # print(label_contribution)
    # print("\n\n")
    
    label_feature_value, prediction_feature_value = extract_label_prediction_feature_values_json(feature_name,
                                                                                                 label_contribution,
                                                                                                 prediction_contribution)
    # print("\ncompute_exact_score_between_2_contributions_json_based ------\n")
    # print(f"feature: {feature_name}")
    # print(f"label value: {label_feature_value}")
    # print(f"prediction value: {prediction_feature_value} \n\n")

    if is_exact_match_json_based(label_feature_value=label_feature_value,
                                 prediction_feature_value=prediction_feature_value):
        return 1
    else:
        return 0


def compute_partial_score_between_2_contributions_json_based(prediction_contribution, label_contribution, feature_name):
    label_feature_value, prediction_feature_value = extract_label_prediction_feature_values_json(feature_name,
                                                                                                 label_contribution,
                                                                                                 prediction_contribution)
    if is_partial_match_json_based(label_feature_value=label_feature_value,
                                   prediction_feature_value=prediction_feature_value):
        return 1
    else:
        return 0


def make_list_of_pairs_json_based(label_list, prediction_list):
    # make list of (label,prediction,similarity)
    list_of_label_prediction_pairs = []
    for label, prediction in zip(label_list, prediction_list):
        pair_list = []
        label_contribution_list = get_contribution_list_json_based(label)
        prediction_contribution_list = get_contribution_list_json_based(prediction)
        for item1 in label_contribution_list:
            for item2 in prediction_contribution_list:
                item1_str = item1
                item2_str = item2
                if isinstance(item1, dict):
                    item1_str = json.dumps(item1)
                if isinstance(item2, dict):
                    item2_str = json.dumps(item2)

                pair_list.append((item1, item2, calculate_fuzz_ratio(item1_str, item2_str)))

        max_selectable_pairs = min(len(label_contribution_list), len(prediction_contribution_list))
        top_similar_pairs = sorted(pair_list, key=lambda x: x[2], reverse=True)[:max_selectable_pairs]
        list_of_label_prediction_pairs.extend(top_similar_pairs)

    return list_of_label_prediction_pairs


def get_contribution_list_json_based(text):
    contribution_list = parse_json_string(text)
    if not isinstance(contribution_list, list):
        contribution_list = [contribution_list]

    return contribution_list
    # if isinstance(text, str):
    #     text = parse_json_string(text)
    # if not isinstance(text, list):
    #     return [text]
    # return text


def get_feature_based_denominator_count_json_based(feature_name, item_list):
    result = 0
    for item in item_list:
        json_item_list = get_contribution_list_json_based(item)
        if is_answerable(json_item_list[0]):
            result += get_number_of_values_of_feature_json_based(feature_name, json_item_list)
    return result

def calculate_feature_based_tp_json_based(pair_list, feature_name, mode):
    result = 0
    for pair in pair_list:
        label, prediction = pair[0], pair[1]
        if mode == MatchType.EXACT.value:
            result += compute_exact_score_between_2_contributions_json_based(prediction_contribution=prediction,
                                                                             label_contribution=label,
                                                                             feature_name=feature_name)
        if mode == MatchType.PARTIAL.value:
            result += compute_partial_score_between_2_contributions_json_based(prediction_contribution=prediction,
                                                                               label_contribution=label,
                                                                               feature_name=feature_name)
    return result


def get_result_dict(exact_f1, exact_precision, exact_recalls, general_accuracy, partial_f1, partial_precision,
                    partial_recalls):
    return {"general_accuracy": general_accuracy, "exact_recalls": exact_recalls,
            "partial_recalls": partial_recalls, "exact_precisions": exact_precision,
            "partial_precisions": partial_precision, "exact_f1s": exact_f1, "partial_f1s": partial_f1}


def calculate_f1(recall, precision):
    if precision + recall != 0:
        return 2 * (precision * recall) / (precision + recall)
    else:
        return 0.0


def get_all_f1(exact_precision, exact_recalls, partial_precision, partial_recalls):
    exact_f1 = {key: calculate_f1(exact_recalls[key], exact_precision[key]) for key in exact_recalls if
                key in exact_precision}
    partial_f1 = {key: calculate_f1(partial_recalls[key], partial_precision[key]) for key in partial_recalls if
                  key in partial_precision}
    return exact_f1, partial_f1


def flatten_result_dict(evaluation_dict):
    result = {}
    for k, v in evaluation_dict.items():
        if isinstance(v, dict):
            for sub_k, sub_v in v.items():
                result[f"{k}_{sub_k}"] = sub_v
        else:
            result[k] = v
    return result


# ----------------------------------------------------------------------------------------------------------------


def recall_json_based(label_list, pair_list, feature_name, mode=MatchType.EXACT.value):
    
    total_items_with_value_for_feature_in_labels = get_feature_based_denominator_count_json_based(feature_name,
                                                                                                  label_list)
   
    total_items_with_correct_value_in_predictions = calculate_feature_based_tp_json_based(pair_list, feature_name, mode)
    # print(feature_name,total_items_with_correct_value_in_predictions)

    if total_items_with_value_for_feature_in_labels == 0:
        return 0
    else:
        recall = total_items_with_correct_value_in_predictions / total_items_with_value_for_feature_in_labels
        assert 0 <= recall <= 1
        # print(mode + " ", feature_name, "recall: ", recall, "\ttp: ",
        #       total_items_with_correct_value_in_predictions, "\tp: ",
        #       total_items_with_value_for_feature_in_labels)
        return recall


# tp/pp
def precision_json_based(prediction_list, pair_list, feature_name, mode=MatchType.EXACT.value):
    total_items_with_value_for_feature_in_predictions = get_feature_based_denominator_count_json_based(feature_name,
                                                                                                       prediction_list)
    total_items_with_correct_value_in_predictions = calculate_feature_based_tp_json_based(pair_list, feature_name, mode)

    if total_items_with_value_for_feature_in_predictions == 0:
        return 0
    else:
        precision = total_items_with_correct_value_in_predictions / total_items_with_value_for_feature_in_predictions
        assert 0 <= precision <= 1
        # print(mode + " ", feature_name, "precision: ", precision, "\ttp: ",
        #       total_items_with_correct_value_in_predictions, "\tpp: ",
        #       total_items_with_value_for_feature_in_predictions)
        return precision


def parse_json_string(data):
    if isinstance(data, (dict, list)):
        return data
    try:
        parsed_json = json.loads(data)
        return parsed_json
    except json.JSONDecodeError:
        return data


def compute_all_metrics_json_based(label_list, prediction_list):
    label_prediction_pairs = make_list_of_pairs_json_based(label_list=label_list, prediction_list=prediction_list)
    
    general_accuracy = Metrics.general_accuracy_json_based(label_list=label_list, prediction_list=prediction_list)
    exact_recalls, partial_recalls = Metrics.all_recall_json_based(label_list=label_list,
                                                                   pair_list=label_prediction_pairs)

    exact_precision, partial_precision = Metrics.all_precision_json_based(prediction_list=prediction_list,
                                                                          pair_list=label_prediction_pairs)

    exact_f1, partial_f1 = get_all_f1(exact_precision, exact_recalls, partial_precision, partial_recalls)

    return get_result_dict(exact_f1, exact_precision, exact_recalls, general_accuracy, partial_f1,
                           partial_precision, partial_recalls)


class Metrics:
    @staticmethod
    # (tp+tn)/p+n
    def general_accuracy_text_based(label_list, prediction_list):
        tp = tn = 0
        for label, prediction in zip(label_list, prediction_list):
            if is_answerable(label) and is_answerable(prediction):
                tp += 1
            if is_unanswerable(label) and is_unanswerable(prediction):
                tn += 1

        accuracy = (tp + tn) / len(label_list)
        assert 0 <= accuracy <= 1
        return accuracy

    @staticmethod
    # (tp+tn)/p+n
    def general_accuracy_json_based(label_list, prediction_list):
        tp = tn = 0
        for label, prediction in zip(label_list, prediction_list):
            label = parse_json_string(label)
            prediction = parse_json_string(prediction)
            if is_answerable(label) and is_answerable(prediction):
                tp += 1
            if is_unanswerable(label) and is_unanswerable(prediction):
                tn += 1
        accuracy = (tp + tn) / len(label_list)
        assert 0 <= accuracy <= 1
        return accuracy


    @staticmethod
    def all_recall_json_based(label_list, pair_list):

        exact_results = {feature_name: recall_json_based(label_list, pair_list, feature_name, MatchType.EXACT.value) for
                         feature_name in feature_names}
        partial_results = {feature_name: recall_json_based(label_list, pair_list, feature_name, MatchType.PARTIAL.value)
                           for
                           feature_name in feature_names}

        exact_results["overall"] = sum(exact_results.values()) / len(exact_results)
        partial_results["overall"] = sum(partial_results.values()) / len(partial_results)

        return exact_results, partial_results


    @staticmethod
    def all_precision_json_based(prediction_list, pair_list):

        exact_results = {
            feature_name: precision_json_based(prediction_list, pair_list, feature_name, MatchType.EXACT.value)
            for feature_name in feature_names}
        partial_results = {
            feature_name: precision_json_based(prediction_list, pair_list, feature_name, MatchType.PARTIAL.value)
            for feature_name in feature_names}

        exact_results["overall"] = sum(exact_results.values()) / len(exact_results)
        partial_results["overall"] = sum(partial_results.values()) / len(partial_results)

        return exact_results, partial_results

    @staticmethod
    def evaluate_property_wise_json_based(label_list, prediction_list):

        evaluation_dict = compute_all_metrics_json_based(label_list, prediction_list)
        return {k: round(v * 100, 4) for k, v in flatten_result_dict(evaluation_dict).items()}

    @staticmethod
    def evaluate_rouge(label_list, prediction_list):

        metric = evaluate.load("rouge")
        result = metric.compute(predictions=prediction_list, references=label_list, use_stemmer=True)
        return {k: round(v * 100, 4) for k, v in result.items()}


    # @staticmethod
    # def evaluate_bleu(label_list, prediction_list):

    #     metric = evaluate.load("bleu")
    #     result = metric.compute(predictions=prediction_list, references=label_list, use_stemmer=True)
    #     return {k: round(v * 100, 4) for k, v in result.items()}


print("Evaluation method defined successfully")

2025-10-09 13:04:26.332184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760015066.711582      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760015066.827479      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Evaluation method defined successfully


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
# Test Mtrics

def test_structure_evaluation():
    labels = "[{ \"geographical area\": \"West african\", \"local name\": \"Salt\", \"common name\": \"Salt\", \"food group\": \"Miscellaneous\", \"food components\": [{ \"label\": \"Salt01_Energy\", \"value\": 0.0, \"unit\": \"Kcal\"} , { \"label\": \"Salt01_Water\", \"value\": 0.5, \"unit\": \"g\"} , { \"label\": \"Salt01_Protein\", \"value\": 0.0, \"unit\": \"g\"} , { \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\"} , { \"label\": \"Salt01_Carbohydrate\", \"value\": 0.0, \"unit\": \"g\"} , { \"label\": \"Salt01_Fibre\", \"value\": 0.0, \"unit\": \"g\"} , { \"label\": \"Salt01_Ash\", \"value\": 99.8, \"unit\": \"g\"} , { \"label\": \"Salt01_Calcium\", \"value\": 216.0, \"unit\": \"mg\"} , { \"label\": \"Salt01_Fer\", \"value\": 1.2, \"unit\": \"mg\"} , { \"label\": \"Salt01_Magensium\", \"value\": 39.0, \"unit\": \"mg\"} , { \"label\": \"Salt01_Phosphore\", \"value\": 166.0, \"unit\": \"mg\"} , { \"label\": \"Salt01_Potassium\", \"value\": 1290.0, \"unit\": \"mg\"} , { \"label\": \"Salt01_Sodium\", \"value\": 38760.0, \"unit\": \"mg\"} , { \"label\": \"Salt01_Zinc\", \""}"

    predictions = "[ fructe \"geographical area\": \"North african\", \"local name\": \"Salt\", \"common name\": \"Salt\", \"food group\": \"Miscellaneous\", \"food components\": [ fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0, \"unit\": \"g\" fructe , fructe \"label\": \"Salt01_Fat\", \"value\": 0.0"

    results = Metrics.evaluate_property_wise_json_based(labels, predictions)
    for k, v in results.items():
        print(f"{k}: {v}")

test_structure_evaluation()

SyntaxError: unmatched '}' (1161675623.py, line 4)

## Import libraries

In [ ]:
# IMPORT =========================================

import os
import random
import json

import numpy as np
import torch
from datasets import DatasetDict, load_dataset
from tokenizers import AddedToken
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, EarlyStoppingCallback
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.optimization import Adafactor, AdafactorSchedule

# from src.util.evaluation_metrics import Metrics

print("\nSuccessfull import")

The code below present code for fine-tuning Flan-T5 Model on knoledge Extraction

## Load Dataset

In [ ]:
# Load dataset

seed = 42
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

num_epoch = 4
run_number = 6
run_id = f"run{run_number}_extraction_epoch{num_epoch}"
path_prefix = "/kaggle/working/"
model_is_from_train_set = 'drop_1'
model_id = "google/flan-t5-large"
max_target_length = 512
max_source_length = 512
label_pad_token_id = -100
max_new_tokens = 512
metric_name = "partial_f1s_overall"
dataset_path = "/kaggle/input/food-contribution-dataset/final_dataset_fixed.jsonl"

# 
dataset = load_dataset("json", data_files=dataset_path)
dataset = dataset.shuffle(seed=seed)

# découpage
dataset = dataset["train"].train_test_split(test_size=0.1, seed=seed)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print("Train example:\n")
print(train_dataset[0])

print("\nTest example:\n")
print(eval_dataset[0])

# # Sauvegarder en JSON
# with open("train_dataset.json", "w", encoding="utf-8") as f:
#     json.dump(train_dataset, f, ensure_ascii=False, indent=2)

# with open("eval_dataset.json", "w", encoding="utf-8") as f:
#     json.dump(eval_dataset, f, ensure_ascii=False, indent=2)

print("Fichiers JSON créés : train_dataset.json et eval_dataset.json")

print("\nDataset loaded successfully")

## Dataset Tokenization

In [ ]:
# Tokenization

fine_tuned_model_repository = f"{path_prefix}/models/json_based/{model_is_from_train_set}"
tokenizer_repository = f"{path_prefix}/models/json_based/{model_is_from_train_set}/tokenizer"
if not os.path.exists(fine_tuned_model_repository):
    os.makedirs(fine_tuned_model_repository)
    os.makedirs(tokenizer_repository)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_tokens(AddedToken("{", normalized=False))
tokenizer.add_tokens(AddedToken("}", normalized=False))
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


# def tokenize_function(sample):
#     # tokenize inputs
#     model_inputs = tokenizer(get_prompt(sample["input"]), max_length=max_source_length, padding="max_length", truncation=True,
#                              return_tensors="pt")

#     # Tokenize targets with the `text_target` keyword argument
#     # Convertir la cible JSON en texte
#     target_text = json.dumps(sample["target"], ensure_ascii=False)
#     labels = tokenizer(text_target=target_text, max_length=max_target_length, padding="max_length",
#                        truncation=True, return_tensors="pt")

#     labels["input_ids"] = [
#         [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]  # type: ignore
#     ]
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

def tokenize_function(batch):
    # Génère le prompt pour chaque entrée
    inputs = [load_prompt(run_number, example) for example in batch["input"]]

    # Convertit chaque target (liste d'objets) en string JSON
    targets = [json.dumps(t, ensure_ascii=False) for t in batch["target"]]

    # Tokenisation des inputs
    model_inputs = tokenizer(
        inputs,
        max_length=max_source_length,
        padding="max_length",
        truncation=True
    )

    # Tokenisation des targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            padding="max_length",
            truncation=True
        )

    # Remplacer pad_token_id par -100 pour ignorer la perte
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# def compute_metrics(eval_preds):

#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]

#     preds = np.where(preds != -100, preds, tokenizer.pad_token_id)  # type: ignore
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # type: ignore
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = Metrics.evaluate_property_wise_json_based(label_list=decoded_labels, prediction_list=decoded_preds) ## JE SUIS ICI
#     result.update(Metrics.evaluate_rouge(label_list=decoded_labels, prediction_list=decoded_preds))
#     return result

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)  # type: ignore
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # pour ameliorerer le decodage
    # decoded_preds = [clean_json_text(p) for p in decoded_preds]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # type: ignore
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # pour ameliorer le decodage
    # decoded_labels = [clean_json_text(l) for l in decoded_labels]

    # Enregistrement des sorties
    with open(f"{run_id}.jsonl", "w") as f:
        for pred, label in zip(decoded_preds, decoded_labels):
            f.write(json.dumps({"prediction": pred, "label": label}) + "\n")

    # Calcul des métriques
    result = Metrics.evaluate_property_wise_json_based(
        label_list=decoded_labels,
        prediction_list=decoded_preds
    )
    result.update(
        Metrics.evaluate_rouge(
            label_list=decoded_labels,
            prediction_list=decoded_preds
        )
    )
    # result.update(
    #     Metrics.evaluate_bleu(
    #         label_list=decoded_labels,
    #         prediction_list=decoded_preds
    #     )
    # )

    # save metric to json
    # Sauvegarde en JSON (fichier)
    with open(f"{run_id}_results.json", "w", encoding="utf-8") as f:
        json.dump(result, f, indent=4, ensure_ascii=False)

    return result


train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
eval_tokenized_dataset = eval_dataset.map(tokenize_function, batched=True)

print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

## Evaluation Methodology

The code below present used metrics to evaluate our Flan-t5 model on knowledge Extraction

## Fine tune

In [ ]:
# Fine tuning

optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10, early_stopping_threshold=0.001)

training_args = Seq2SeqTrainingArguments(
    output_dir=fine_tuned_model_repository,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=num_epoch,
    generation_max_length=max_new_tokens,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    greater_is_better=True,
    logging_dir=f"{fine_tuned_model_repository}/logs",
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    seed=seed
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
    eval_dataset=eval_tokenized_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),
    callbacks=([early_stopping_callback])
)

trainer.train()
tokenizer.save_pretrained(tokenizer_repository)
best_ckpt_path = trainer.state.best_model_checkpoint
print(f"best epoch: {best_ckpt_path}")

save_training_curves_by_epoch(trainer, run_id)

In [ ]:
import matplotlib.pyplot as plt
import os

def save_training_curves_by_epoch(trainer, run_id, output_dir="training_curves"):
    """
    Enregistre les courbes d'entraînement et d'évaluation au format SVG par epoch.
    
    Args:
        trainer : instance de Seq2SeqTrainer
        run_id : identifiant unique pour préfixer les fichiers
        output_dir : dossier où sauvegarder les SVG
    """
    os.makedirs(output_dir, exist_ok=True)

    # Récupérer l'historique du trainer
    logs = trainer.state.log_history

    # Extraire les courbes par epoch
    epoch_losses = {}
    for log in logs:
        if 'epoch' in log:
            epoch = log['epoch']
            for k, v in log.items():
                if k not in ['step', 'epoch', 'learning_rate']:
                    if k not in epoch_losses:
                        epoch_losses[k] = []
                    epoch_losses[k].append((epoch, v))

    # Plot Loss
    if 'loss' in epoch_losses:
        epochs, values = zip(*epoch_losses['loss'])
        plt.figure(figsize=(8,5))
        plt.plot(epochs, values, label='Train Loss')
        if 'eval_loss' in epoch_losses:
            epochs_eval, eval_values = zip(*epoch_losses['eval_loss'])
            plt.plot(epochs_eval, eval_values, label='Eval Loss')
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training and Evaluation Loss per Epoch")
        plt.legend()
        plt.grid(True)
        loss_path = os.path.join(output_dir, f"{run_id}_loss_epoch.svg")
        plt.savefig(loss_path, format='svg')
        plt.close()
        print(f"Loss curve per epoch saved to {loss_path}")

    # Plot autres métriques
    metric_keys = [k for k in epoch_losses.keys() if k not in ['loss','eval_loss']]
    for key in metric_keys:
        epochs, values = zip(*epoch_losses[key])
        plt.figure(figsize=(8,5))
        plt.plot(epochs, values, label=key)
        plt.xlabel("Epoch")
        plt.ylabel(key)
        plt.title(f"Metric: {key} per Epoch")
        plt.legend()
        plt.grid(True)
        metric_path = os.path.join(output_dir, f"{run_id}_{key}_epoch.svg")
        plt.savefig(metric_path, format='svg')
        plt.close()
        print(f"Metric curve per epoch saved to {metric_path}")

print("\nSave plot function defined successfully")

## Inference

In [ ]:
# 1. Recharger le modèle et le tokenizer sauvegardés
model = AutoModelForSeq2SeqLM.from_pretrained(best_ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_repository)

trainer.model = model

# 3. Générer des prédictions
predictions = trainer.predict(eval_tokenized_dataset)
preds = predictions.predictions

# Décoder les prédictions
decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

# Décoder les labels
labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# 5. Sauvegarder en JSONL
output_file = f"predictions_run{run_number}.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for pred, label in zip(decoded_preds, decoded_labels):
        json.dump({"prediction": pred, "label": label}, f, ensure_ascii=False)
        f.write("\n")

print(f"Fichier JSONL sauvegardé: {output_file}")

# KE4KI pipeline ---------------------------------------------------------------------------------------

## 1. Load test papers

In [ ]:
west_african_food_composition_paper_dir = "/kaggle/input/papers-ke4ki-test/ca7779b.PDF"
tunisia_food_composition_paper_dir = "/kaggle/input/papers-ke4ki-test/tunisia food composition table 1 - 2007.pdf"

# choose paper for the test
choosen_paper_dir = tunisia_food_composition_paper_dir

print("Test papers loaded successfully")

## 2. Divide the choosen paper into pages, add prompt template and tokenize data

In [ ]:
# code ...
import pdfplumber
import json
import re
from collections import defaultdict

def extract_pdf_pages(pdf_path):
    segments = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                segments.append({
                    "page": i + 1,
                    "text": text.strip()
                })
    return segments

pages = extract_pdf_pages(choosen_paper_dir)

print("\nThe choosen paper has been divided into pages\n")

print(pages[20])

In [ ]:
# IMPORT =========================================

import os
import random
import json

import numpy as np
import torch
from datasets import DatasetDict, load_dataset
from tokenizers import AddedToken
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, EarlyStoppingCallback
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.optimization import Adafactor, AdafactorSchedule

# from src.util.evaluation_metrics import Metrics

print("\nSuccessfull import")

## 3. Load the model and tokenizer

In [ ]:
# code ...
tokenizer_dir = "/kaggle/working/models/json_based/drop_1/tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
tokenizer.add_tokens(AddedToken("{", normalized=False))
tokenizer.add_tokens(AddedToken("}", normalized=False))
model_checkpoint = "/kaggle/working/models/json_based/drop_1/checkpoint-258"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Si GPU dispo
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

print("\nModel loaded successfully\n")

## 4. Perform extraction and validate model output

In [ ]:
# code ...
from json_repair import repair_json

def predict_one(text, max_new_tokens=512, num_beams=4):
    """
    Prend un seul texte en entrée et retourne la prédiction.
    """
    # Tokenisation
    inputs = tokenizer(
        load_prompt(6, text),
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(device)

    # Génération
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams
        )

    # Conversion en numpy et nettoyage comme dans compute_metrics
    preds = outputs.cpu().numpy()
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    # Décodage cohérent avec compute_metrics
    decoded_pred = tokenizer.decode(preds[0], skip_special_tokens=True)

    return decoded_pred

predictions = []

# Limit the list to use only one page (Page 20 for example)
page_test = []
page_test.append(pages[20])
for p in page_test:
    prediction = predict_one(p.get("text"))
    predictions.append({
        "paper": choosen_paper_dir,
        "page":p.get("page"),
        "extracted_data": repair_json(prediction)
    })

print("\nExtraction finished successfully\n")
print(predictions[:2])

## 5. Index extracted items into orkg and save links

In [ ]:
# code ...

from orkg import ORKG, Hosts
from difflib import get_close_matches
import time
import json

print("\nNecessary libs imported !\n")


In [ ]:


# -----------------------------------------
# Connexion au sandbox ORKG
# -----------------------------------------
sandbox_url = "https://sandbox.orkg.org"
orkg = ORKG(host=Hosts.SANDBOX, creds=('emmanuelleuna758@gmail.com', 'Leuna75820u2698@'))
orkg_resources_manager = orkg.resources
orkg_statements_manager = orkg.statements
orkg_litterals_manager = orkg.literals

# -----------------------------------------
# Get template properties
# -----------------------------------------
TEMPLATE_ID = "R677398"
TEMPLATE_PROPERTIES = [
    # Food template --------------------------
    {
        "label": "Food template",
        "propertyShape": "R677462",
        "propertyShapePath": "P142021",
        "propertyShapePathLabel": "food type"
    },
    {
        "label": "Food template",
        "propertyShape": "R677459",
        "propertyShapePath": "P142023",
        "propertyShapePathLabel": "scientific name"
    },
    {
        "label": "Food template",
        "propertyShape": "R677464",
        "propertyShapePath": "P20098",
        "propertyShapePathLabel": "has description"
    },
    {
        "label": "Food template",
        "propertyShape": "R677465",
        "propertyShapePath": "P135009",
        "propertyShapePathLabel": "geographical area"
    },
    {
        "label": "Food template",
        "propertyShape": "R677457",
        "propertyShapePath": "P142020",
        "propertyShapePathLabel": "local name"
    },
    {
        "label": "Food template",
        "propertyShape": "R677461",
        "propertyShapePath": "P142022",
        "propertyShapePathLabel": "food form"
    },
    {
        "label": "Food template",
        "propertyShape": "R677458",
        "propertyShapePath": "P142024",
        "propertyShapePathLabel": "common name"
    },
    {
        "label": "Food template",
        "propertyShape": "R1361187",
        "propertyShapePath": "P142026",
        "propertyShapePathLabel": "food image"
    },
    {
        "label": "Food template",
        "propertyShape": "R707241",
        "propertyShapePath": "P151019",
        "propertyShapePathLabel": "eaten with"
    },
    {
        "label": "Food template",
        "propertyShape": "R707751",
        "propertyShapePath": "P35126",
        "propertyShapePathLabel": "Group"
    },
    {
        "label": "Food template",
        "propertyShape": "R677460",
        "propertyShapePath": "P6003",
        "propertyShapePathLabel": "has ingredient"
    },
    {
        "label": "Food template",
        "propertyShape": "R707240",
        "propertyShapePath": "P62002",
        "propertyShapePathLabel": "food group"
    },
    {
        "label": "Food template",
        "propertyShape": "R677466",
        "propertyShapePath": "P62073",
        "propertyShapePathLabel": "food component"
    },
    
    # Food component template --------------------------
    {
        "label": "Food component template",
        "propertyShape": "R684340",
        "propertyShapePath": ":P62093",
        "propertyShapePathLabel": "food component name"
    },
    {
        "label": "Food component template",
        "propertyShape": "R684342",
        "propertyShapePath": ":P5086",
        "propertyShapePathLabel": "value"
    },
    
    # Quantity Value template --------------------------
    {
        "label": "Quantity Value",
        "propertyShape": "R172467",
        "propertyShapePath": ":P45075",
        "propertyShapePathLabel": "numericValue"
    },
    {
        "label": "Quantity Value",
        "propertyShape": "R172468",
        "propertyShapePath": ":P45076",
        "propertyShapePathLabel": "unit"
    },
]

# -----------------------------------------
# Functions
# -----------------------------------------

# Find or add resource
def find_or_add_resource(label, instanceOf):
    
    r = orkg.resources.get_unpaginated(q=label,exact=True, include=instanceOf)
    r = r.content
   
    if(len(r)>0):
        return r[0]
    else:
        # print(instanceOf)
        r = orkg_resources_manager.add(label=label, classes=instanceOf)
        print('Resource ' + label+ " added")
        return r.content

# Get predicate id from our template
def get_predicate_id(predicate_string):
    predicate_id = None
    for item in TEMPLATE_PROPERTIES:
        if item['propertyShapePathLabel'] == predicate_string:
            predicate_id = item['propertyShapePath']
            return predicate_id

# add statementitem
def add_statement(predicate_string, subject_id, object_classes, extracted_item, object_id=None):
    predicate_found = 0
    if(object_id == None):
        extracted_item = dict(extracted_item)
        label_column = get_close_matches(predicate_string, extracted_item.keys(), n=1, cutoff=cutoff)
        if(len(label_column)>0):
            predicate_found = 1
            label = label_column[0]
            value = extracted_item[label]
            object_resource = find_or_add_resource(label=value, instanceOf=object_classes)
            object_id = object_resource['id']
            
    if(predicate_found == 1):
        
        # -----------------------------------------
        # add {predicate_string} statement
        # -----------------------------------------
        # get statements by subject and predicate without paggination
        statements_subject_predicate = orkg_statements_manager.get_by_subject_and_predicate_unpaginated(subject_id, get_predicate_id(predicate_string))
        statements_subject_predicate = statements_subject_predicate.content
        # get statements by predicate and object without paggination
        statements_object_predicate = orkg_statements_manager.get_by_object_and_predicate_unpaginated(object_id, get_predicate_id(predicate_string))
        statements_object_predicate = statements_object_predicate.content
        # check if statement already exist
        if(statements_subject_predicate.__len__() == 0 | statements_object_predicate.__len__() == 0):
            print('The statement '+subject_id+' '+predicate_string+' '+object_id +' does not exist')
            print('Adding of the statement ...')
            # add statement
            orkg_statements_manager.add(subject_id=subject_id, predicate_id=get_predicate_id(predicate_string), object_id=object_id)
        else:
            # Obtenir les IDs de chaque liste
            ids_liste2 = {item["id"] for item in statements_object_predicate}

            # Intersection : éléments de liste1 dont l'id est dans liste2
            intersection = [item for item in statements_subject_predicate if item["id"] in ids_liste2]
            if(intersection.__len__() == 0):
                print('The statement '+subject_id+' '+predicate_string+' '+object_id +' does not exist')
                print('Adding of the statement ...')
                # add statement
                orkg_statements_manager.add(subject_id=subject_id, predicate_id=get_predicate_id(predicate_string), object_id=object_id)
            else: 
                print('The statement '+subject_id+' '+predicate_string+' '+object_id +' already exist')
                print('skipping of the statement ...')


# -----------------------------------------
# Get extracted data
# -----------------------------------------
all_pred = predictions
    

# Fusionner toutes les prédictions en une seule liste
merged_predictions = []

for item in all_pred:
    predictions_temp = item.get("extracted_data", [])
    print(predictions_temp)
    predictions_temp = json.loads(predictions_temp)
    # Ajoute chaque prédiction à la liste globale
    for pred in predictions_temp:
        merged_predictions.append(pred)

data = merged_predictions

# data = [
#   {
#     "geographical area": "West african",
#     "local name": "Salt",
#     "common name": "Salt",
#     "food group": "Miscellaneous",
#     "food components": [
#       { "label": "Salt01_Energy", "value": 0.0, "unit": "Kcal" },
#       { "label": "Salt01_Water", "value": 0.5, "unit": "g" },
#       { "label": "Salt01_Protein", "value": 0.0, "unit": "g" },
#       { "label": "Salt01_Fat", "value": 0.0, "unit": "g" },
#       { "label": "Salt01_Carbohydrate", "value": 0.0, "unit": "g" },
#       { "label": "Salt01_Fibre", "value": 0.0, "unit": "g" },
#       { "label": "Salt01_Ash", "value": 99.8, "unit": "g" },
#       { "label": "Salt01_Calcium", "value": 216.0, "unit": "mg" },
#       { "label": "Salt01_Fer", "value": 1.2, "unit": "mg" },
#       { "label": "Salt01_Magnesium", "value": 39.0, "unit": "mg" },
#       { "label": "Salt01_Phosphore", "value": 166.0, "unit": "mg" },
#       { "label": "Salt01_Potassium", "value": 1290.0, "unit": "mg" },
#       { "label": "Salt01_Sodium", "value": 38760.0, "unit": "mg" },
#       { "label": "Salt01_Zinc", "value": 0.1, "unit": "mg" }
#     ]
#   }
# ]

# Start the timer
start = time.time()

# -----------------------------------------
# Map extracted data
# -----------------------------------------
mapped_data = []
cutoff = 0.90
for item in data:
    properties_found = []
    for property in TEMPLATE_PROPERTIES:
        label_found = get_close_matches(property["propertyShapePathLabel"], item.keys(), n=1, cutoff=cutoff)
        
        if(len(label_found) >0):
            properties_found.append(label_found[0])
            
    # get value in item for each property foud
    item_properties = {}
    for prop in properties_found:
        item_properties[prop] = item[prop]
    
    # add mapped item to mapped_data
    mapped_data.append(item_properties)


import json


indexed_statement = []
resources_urls = []
for item in mapped_data:
    # -----------------------------------------
    # Create or get the general ressource (Food)
    # -----------------------------------------
    nb_statement = 0
    item = dict(item)
    label_column = get_close_matches("local name", item.keys(), n=1, cutoff=cutoff)
    if(label_column.__len__() > 0):
        label = label_column[0]
        value = item[label]
        general_resource = find_or_add_resource(label=value, instanceOf=['C34019'])
        resource_url = f"https://sandbox.orkg.org/resources/{general_resource['id']}?noRedirect"
        
        # -----------------------------------------
        # add local name statement
        # -----------------------------------------
        # Here the object is also the subject
        add_statement(predicate_string="local name", subject_id=general_resource['id'], object_classes=['C34019'], extracted_item=item)
        nb_statement += 1
        
        # -----------------------------------------
        # add scientific name statement
        # -----------------------------------------
        if( get_close_matches("local name", item.keys(), n=1, cutoff=cutoff).__len__() >0):
            add_statement("scientific name", general_resource['id'], ['C86010'], item)
            nb_statement += 1
        
        # -----------------------------------------
        # add common name statement
        # -----------------------------------------
        if(get_close_matches("common name", item.keys(), n=1, cutoff=cutoff).__len__() >0):
            add_statement("common name", general_resource['id'], ['C34019'], item)
            nb_statement += 1
        
        # -----------------------------------------
        # add food group statement
        # -----------------------------------------
        if(get_close_matches("food group", item.keys(), n=1, cutoff=cutoff).__len__() >0):
            add_statement("food group", general_resource['id'], ['C34000'], item)
            nb_statement += 1
        
        # -----------------------------------------
        # add geographical area statement
        # -----------------------------------------
        if(get_close_matches("geographical area", item.keys(), n=1, cutoff=cutoff).__len__() >0):
            add_statement("geographical area", general_resource['id'], ['C86012'], item)
            nb_statement += 1
        
        # -----------------------------------------
        # add food form statement
        # -----------------------------------------
        if(get_close_matches("food form", item.keys(), n=1, cutoff=cutoff).__len__() >0):
            add_statement("food form", general_resource['id'], ['C86013'], item)
            nb_statement += 1
        
        # -----------------------------------------
        # add food components statement
        # -----------------------------------------
        if(get_close_matches("food components", item.keys(), n=1, cutoff=cutoff).__len__() >0):
            for component in item['food components']:
                component = dict(component)
                c = find_or_add_resource(label=item['local name']+'_'+ component['label'],instanceOf=["C34009"])                
                # add food component name statement
                
                if( "label" in component.keys()):
                    add_statement("food component name", c['id'], ['C34009'], {
                        "food component name": component["label"]
                    })
                    
                # add food component value statement C23008
                if( "value" in component.keys() and "unit" in component.keys()):
                    v = find_or_add_resource(label=str(component['value'])+' '+ str(component['unit']),instanceOf=["C23008"])

                    # add numericValue value statement
                    l = orkg_litterals_manager.get_all(q=component["value"], exact=True)
                    l = l.content
                    if(len(l)==0 ):
                        l = orkg_litterals_manager.add(label=component["value"], datatype='xsd:decimal')
                        l = l.content
                    else:
                        l = l[0]
                    add_statement("numericValue", v['id'], ['literal'], None, l['id'])

                    # add unit value statement
                    add_statement("unit", v['id'], ['C23009'], component)
                    
                    add_statement("value", c['id'], ['C23008'], None, v['id'])
                    
                    
                add_statement("food components", general_resource["id"], ['C34009'], None, c["id"])
                nb_statement += 1
        
        # -----------------------------------------
        # add food ingredient statement
        # -----------------------------------------
        if( "ingredients" in item.keys()):
            for ingredient in item['ingredients']:
                ingredient = dict(ingredient)
                i = find_or_add_resource(label=item['local name']+'_ingredient'+ ingredient['label'], instanceOf="C77007")
                
                # add food ingredient common name statement
                if( "common name" in ingredient.keys()):
                    add_statement("common name", i['id'], ['C34019'], ingredient)
                    
                # add food ingredient scientific name statement
                if( "scientific name" in ingredient.keys()):
                    add_statement("scientific name", i['id'], ['C86010'], ingredient)
                    
                add_statement("has ingredient", general_resource["id"], ['C77007'], None, i["id"])
                nb_statement += 1
                
        # get the resource statements
        general_ressources_statements = orkg_statements_manager.get_by_subject_unpaginated(general_resource['id'])
        general_ressources_statements = general_ressources_statements.content
        indexed_statement.append(general_ressources_statements)
        
        # print statement list
        print(f"\n\nnumber of statements:  {general_ressources_statements.__len__()}")
        
        # adding resource url
        resources_urls.append({
            "label":value,
            "url": resource_url
        })

        print(f'URL: {resource_url} ')
    else:
        print('This ressource can not be index')


json.dump(resources_urls, open('resources_urls_wfct.json', 'w'), indent=4)
# -----------------------------------------
# 
# -----------------------------------------

# Stop the timer
end = time.time()
print(f"Execution time : {end - start:.4f} seconds")

## 